#decision-trees


In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.tree import DecisionTreeClassifier
from numpy import log2 as log
from sklearn.datasets import make_gaussian_quantiles
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
PassengerId = test['PassengerId']
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
import re

In [4]:
original_train = train.copy() 
full_data = [train, test]
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())


for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset.loc[np.isnan(dataset['Age']), 'Age'] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)

def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)

for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    title_mapping = {"Mr": 1, "Master": 2, "Mrs": 3, "Miss": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] ;

In [5]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Survived']
train_data = train.drop(drop_elements, axis = 1)
train_data['Survived']=train['Survived']

In [6]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
test_data  = test.drop(drop_elements, axis = 1)

In [7]:
class TreeNode:
    def __init__(self, data,output):
        self.data = data
        self.children = {}
        self.output = output
        self.index = -1
    def add_child(self,feature_value,obj):
        self.children[feature_value] = obj

In [8]:
class decision_tree_classifier:
    def __init__(self):
        self.__root = None

    def __count_unique(self,Y):
        d = {}
        for i in Y:
            if i not in d:
                d[i]=1
            else:
                d[i]+=1
        return d


    def __entropy(self,Y):
        freq_map = self.__count_unique(Y)
        entropy_ = 0
        total = len(Y)
        for i in freq_map:
            p = freq_map[i]/total
            entropy_ += (-p)*math.log2(p)
        return entropy_

    def __gain_ratio(self,X,Y,selected_feature):
        info_orig = self.__entropy(Y)
        info_f = 0  
        split_info = 0
        values = set(X[:,selected_feature])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y
        initial_size = df.shape[0] 
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            info_f += (current_size/initial_size)*self.__entropy(df1[df1.shape[1]-1])
            split_info += (-current_size/initial_size)*math.log2(current_size/initial_size)
        if split_info == 0 :
            return math.inf

        info_gain = info_orig - info_f
        gain_ratio = info_gain / split_info
        return gain_ratio

    def __gini_index(self,Y):
        freq_map = self.__count_unique(Y)
        gini_index_ = 1
        total = len(Y)
        for i in freq_map:
            p = freq_map[i]/total
            gini_index_ -= p**2
        return gini_index_

    def __gini_gain(self,X,Y,selected_feature):
        gini_orig = self.__gini_index(Y)
        gini_split_f = 0 
        values = set(X[:,selected_feature])
        df = pd.DataFrame(X) 
        df[df.shape[1]] = Y
        initial_size = df.shape[0] 
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            gini_split_f += (current_size/initial_size)*self.__gini_index(df1[df1.shape[1]-1])

        gini_gain_ = gini_orig - gini_split_f
        return gini_gain_


    def __decision_tree(self,X,Y,features,level,metric,classes):
        if len(set(Y)) == 1:
            print("Level",level)
            output = None
            for i in classes:
                if i in Y:
                    output = i
                    print("Count of",i,"=",len(Y))
                else :
                    print("Count of",i,"=",0)
            if metric == "gain_ratio":
                print("Current Entropy is =  0.0")
            elif metric == "gini_index":
                print("Current Gini Index is =  0.0")

            print("Reached leaf Node")
            print()
            return TreeNode(None,output)

        if len(features) == 0:
            print("Level",level)
            freq_map = self.__count_unique(Y)
            output = None
            max_count = -math.inf
            for i in classes:
                if i not in freq_map:
                    print("Count of",i,"=",0)
                else :
                    if freq_map[i] > max_count :
                        output = i
                        max_count = freq_map[i]
                    print("Count of",i,"=",freq_map[i])

            if metric == "gain_ratio":
                print("Current Entropy  is =",self.__entropy(Y))
            elif metric == "gini_index":
                print("Current Gini Index is =",self.__gini_index(Y))            

            print("Reached leaf Node")
            print()
            return TreeNode(None,output)

        max_gain = -math.inf
        final_feature = None
        for f in features :
            if metric == "gain_ratio":
                current_gain = self.__gain_ratio(X,Y,f)
            elif metric =="gini_index":
                current_gain = self.__gini_gain(X,Y,f)

            if current_gain > max_gain:
                max_gain = current_gain
                final_feature = f

        print("Level",level)
        freq_map = self.__count_unique(Y)
        output = None
        max_count = -math.inf

        for i in classes:
            if i not in freq_map:
                print("Count of",i,"=",0)
            else :
                if freq_map[i] > max_count :
                    output = i
                    max_count = freq_map[i]
                print("Count of",i,"=",freq_map[i])

        if metric == "gain_ratio" :        
            print("Current Entropy is =",self.__entropy(Y))
            print("Splitting on feature  X[",final_feature,"] with gain ratio ",max_gain,sep="")
            print()
        elif metric == "gini_index":
            print("Current Gini Index is =",self.__gini_index(Y))
            print("Splitting on feature  X[",final_feature,"] with gini gain ",max_gain,sep="")
            print()
            
        unique_values = set(X[:,final_feature])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y

        current_node = TreeNode(final_feature,output)
        index  = features.index(final_feature)
        features.remove(final_feature)
        for i in unique_values:
            df1 = df[df[final_feature] == i]
            node = self.__decision_tree(df1.iloc[:,0:df1.shape[1]-1].values,df1.iloc[:,df1.shape[1]-1].values,features,level+1,metric,classes)
            current_node.add_child(i,node)    
        features.insert(index,final_feature)

        return current_node
    
    def fit(self,X,Y,metric="gain_ratio"):
        features = [i for i in range(len(X[0]))]
        classes = set(Y)
        level = 0
        if metric != "gain_ratio" :
            if metric != "gini_index":
                metric="gain_ratio" 
        self.__root = self.__decision_tree(X,Y,features,level,metric,classes)
        
    def __predict_for(self,data,node):
        if len(node.children) == 0 :
            return node.output

        val = data[node.data]      
        if val not in node.children :
            return node.output
        
        return self.__predict_for(data,node.children[val])

    def predict(self,X):
        Y = np.array([0 for i in range(len(X))])
        for i in range(len(X)):
            Y[i] = self.__predict_for(X[i],self.__root)
        return Y
    
    def score(self,X,Y):
        Y_pred = self.predict(X)
        count = 0
        for i in range(len(Y_pred)):
            if Y_pred[i] == Y[i]:
                count+=1
        return count/len(Y_pred)

In [9]:
clf1 = decision_tree_classifier() 
clf1.fit(train_data.drop(columns=['Survived'], axis=1).values,train_data['Survived'].values)
Y_pred = clf1.predict(test_data.values)

Level 0
Count of 0 = 549
Count of 1 = 342
Current Entropy is = 0.9607079018756469
Splitting on feature  X[1] with gain ratio 0.23249201788302346

Level 1
Count of 0 = 81
Count of 1 = 233
Current Entropy is = 0.8236550739295191
Splitting on feature  X[0] with gain ratio 0.135485323167635

Level 2
Count of 0 = 3
Count of 1 = 91
Current Entropy is = 0.20390587550482003
Splitting on feature  X[7] with gain ratio 0.04893942058911155

Level 3
Count of 0 = 1
Count of 1 = 33
Current Entropy is = 0.19143325481419343
Splitting on feature  X[3] with gain ratio inf

Level 4
Count of 0 = 1
Count of 1 = 33
Current Entropy is = 0.19143325481419343
Splitting on feature  X[8] with gain ratio inf

Level 5
Count of 0 = 1
Count of 1 = 33
Current Entropy is = 0.19143325481419343
Splitting on feature  X[4] with gain ratio 0.09490752723995388

Level 6
Count of 0 = 1
Count of 1 = 6
Current Entropy is = 0.5916727785823275
Splitting on feature  X[5] with gain ratio 0.20108786369811496

Level 7
Count of 0 = 0
Co

In [10]:
print("Predictions :",Y_pred)

Predictions : [0 1 0 0 0 0 1 0 1 0 0 1 1 0 1 1 0 0 0 1 0 1 1 0 1 0 1 0 1 0 0 0 0 0 1 0 0
 0 0 1 0 1 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 1 0 1 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 1 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0
 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 0
 0 0 1 0 0 0 1 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 1]


In [11]:
print("Score :",clf1.score(train_data.drop(columns=['Survived'], axis=1).values,train_data['Survived'].values))

Score : 0.9012345679012346


In [12]:
ans=pd.read_csv('/content/ans.csv')

In [13]:
from sklearn.metrics import accuracy_score

In [14]:
accuracy_score(ans['Survived'], Y_pred)

0.8038277511961722

#RandomForest

In [15]:

class RandomForest:
    def __init__(self, trees, n_trees, max_feature ,
                 prediction_aggrigation_calculation):
        self.n_estimators = n_trees
        self.max_features = max_feature
        self.tree_feature_indexes = []
        self.prediction_aggrigation_calculation = prediction_aggrigation_calculation 
        self.trees = trees

    def _make_random_suset(self, X, y, n_subsets, replasment=True):
        subset = []
        sample_size = (X.shape[0] if replasment else (X.shape[0] // 2))
        Xy = np.concatenate((X, y), axis=1)
        np.random.shuffle(Xy)
        for i in range(n_subsets):
            index = np.random.choice(range(sample_size), size=np.shape(range(sample_size)), replace=replasment)
            X = Xy[index][:, :-1]
            y = Xy[index][: , -1]
            subset.append({"X" : X, "y": y})
        return subset

    def fit(self, X, y):
        n_features = X.shape[1]
        if self.max_features == None:
            self.max_features = int(math.sqrt(n_features))
        subsets = self._make_random_suset(X, y, self.n_estimators)

        for i, subset in enumerate(subsets):
            X_subset , y_subset = subset["X"], subset["y"]
            idx = np.random.choice(range(n_features), size=self.max_features, replace=True)
            self.tree_feature_indexes.append(idx)
            X_subset = X_subset[:, idx]
            y_subset = np.expand_dims(y_subset, axis =1)
            self.trees[i].fit(X_subset, y_subset)

    def predict(self, test_X):
        y_preds = np.empty((test_X.shape[0], self.n_estimators))
        for i, tree in enumerate(self.trees):
            features_index = self.tree_feature_indexes[i]
            X_selected_features = test_X[:, features_index]
            if isinstance(tree, DecisionTreeClassifier):
                y_preds[:, i] = tree.predict(X_selected_features).reshape((-1,))
            else:
                y_preds[:, i] = tree.predict(X_selected_features)
        y_pred = self.prediction_aggrigation_calculation(y_preds)
        return y_pred



In [16]:
class RandomForestClassifier(RandomForest):
    def __init__(self, max_feature, max_depth, n_trees=100, min_sample_split=1.0, min_impurity=1e-7):
        self.prediction_aggrigation_calculation = self._maximum_vote_calculation
        self.trees = []
        for _ in range(n_trees):
          self.trees.append(DecisionTreeClassifier(min_samples_split=min_sample_split,
                                                   min_impurity_decrease=min_impurity, 
                                                   max_depth=max_depth))
        super().__init__(trees=self.trees, n_trees=n_trees,
                         max_feature=max_feature,
                         prediction_aggrigation_calculation=self.prediction_aggrigation_calculation)
    
    def _maximum_vote_calculation(self, y_preds):
        y_pred = np.empty((y_preds.shape[0], 1))
        for i, sample_predictions in enumerate(y_preds):
            y_pred[i] = np.bincount(sample_predictions.astype('int')).argmax()

        return y_pred

In [17]:
y=train_data['Survived'].values
y = y[:, np.newaxis]

In [24]:
param = {
    "n_neibours" : 5
}
random_forest_cla = RandomForestClassifier(n_trees=3, max_feature=2, min_sample_split=1.0, max_depth=45, min_impurity=1e-7)

random_forest_cla.fit(train_data.drop(columns=['Survived'],axis=1).values, y) 

y_pred = random_forest_cla.predict(train_data.drop(columns=['Survived'],axis=1).values)

acc = np.sum(y==y_pred)/train_data.shape[0]
print("Accuracy of the prediction is {}".format(acc))

Accuracy of the prediction is 0.7104377104377104


In [25]:
y_test_pred = random_forest_cla.predict(test_data.values)
accuracy_score(ans['Survived'], y_test_pred)

0.7822966507177034

#adaboost

In [20]:
class AdaBoost:
    def __init__(self):
        self.stumps = None
        self.stump_weights = None
        self.errors = None
        self.sample_weights = None

    def _check_X_y(self, X, y):
        assert set(y) == {1, 0}
        return X, y

def fit(self, X: np.ndarray, y: np.ndarray, iters: int):
    X, y = self._check_X_y(X, y)
    n = X.shape[0]
    self.sample_weights = np.zeros(shape=(iters, n))
    self.stumps = np.zeros(shape=iters, dtype=object)
    self.stump_weights = np.zeros(shape=iters)
    self.errors = np.zeros(shape=iters)

    self.sample_weights[0] = np.ones(shape=n) / n

    for t in range(iters):
        curr_sample_weights = self.sample_weights[t]
        stump = DecisionTreeClassifier(max_depth=1, max_leaf_nodes=2)
        stump = stump.fit(X, y, sample_weight=curr_sample_weights)
        stump_pred = stump.predict(X)
        err = curr_sample_weights[(stump_pred != y)].sum()# / n
        stump_weight = np.log((1 - err) / err) / 2

        new_sample_weights = curr_sample_weights * np.exp(-stump_weight * y * stump_pred)
        new_sample_weights /= new_sample_weights.sum()
        if t+1 < iters:
            self.sample_weights[t+1] = new_sample_weights

        self.stumps[t] = stump
        self.stump_weights[t] = stump_weight
        self.errors[t] = err

    return self


In [21]:
def predict(self, X):
    stump_preds = np.array([stump.predict(X) for stump in self.stumps])
    return np.sign(np.dot(self.stump_weights, stump_preds))

In [22]:
AdaBoost.fit = fit
AdaBoost.predict = predict
clf = AdaBoost().fit(train_data.drop(columns=['Survived'],axis=1).values,train_data['Survived'].values, iters=11)
train_err = (clf.predict(train_data.drop(columns=['Survived'],axis=1).values) != train_data['Survived'].values).mean()
print(f'Train error: {train_err:.1%}')

Train error: 21.8%


In [23]:
y_ad_test_pred = clf.predict(test_data.values)
accuracy_score(ans['Survived'], y_ad_test_pred)

0.937799043062201